Bellow is a model that trains on trafic data more precisely given num_time_step previous speeds it predicts the future forecast_horizion speeds, at the nodes. This is done by combaning temporal convolution over fixed nodes and graph convolution for fixed time intervals, this is combine to a fully connected linear layer to give us our predictions.

Initalize

In [16]:

import torch
import typing
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch_geometric.nn import GCNConv
from torch_geometric.nn import ChebConv
from torch.utils.data import Dataset, DataLoader

CUDA

In [17]:
device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
torch.cuda.empty_cache

Using cuda:0 device


<function torch.cuda.memory.empty_cache() -> None>

Hyper parameteres:

In [18]:
learning_rate = 0.001
criterion = nn.L1Loss(reduction='mean')
num_epochs = 1000
batch_size=16
num_time_steps=4*8
num_features=1
forecast_horizon=1
split=3
multi_horizon = False
patience = 10
Chebyshev = True
Attention =False



Sampling roads, picking nodes.

In [19]:
route_distances = pd.read_csv((r"C:\Users\necad\OneDrive\Desktop\Dataset\PeMSD7_W_228.csv"), header=None).to_numpy()
speeds_array = pd.read_csv((r"C:\Users\necad\OneDrive\Desktop\Dataset\PeMSD7_V_228.csv"), header=None).to_numpy()
sample_routes = [
    0,1, 4,7,8,11,15,108,109,114,115,118,120,123,124,126,127,129,130,132,133,136,139,144,147,216]
sample_routes=[4*i for i in range(57)]
route_distances = route_distances[np.ix_(sample_routes, sample_routes)]
print(route_distances.shape)
speeds_array = speeds_array[:, sample_routes]
mean,std= speeds_array.mean(axis=0), speeds_array.std(axis=0)
#mean,std= np.round(mean,2), np.round(std,2)
#print(f"Mean is {mean} and std is {std}")
#peeds_array=np.random.random(size=(10000,57,2))
#print(speeds_array.shape)
num_feature=num_features


(57, 57)


Adjacency matrix constructor which calculates the Distances to construct a graph.

In [20]:
def compute_adjacency_matrix(
    route_distances: np.ndarray, sigma2: float, epsilon: float
):
    num_routes = route_distances.shape[0]
    route_distances = route_distances / 10000.0
    w2, w_mask = (
        route_distances * route_distances,
        np.ones([num_routes, num_routes]) - np.identity(num_routes),
    )
    return (np.exp(-w2 / sigma2) >= epsilon) * w_mask+np.eye(num_routes)

def adjacency_to_edge_index(adj):
    adj = torch.tensor(adj, dtype=torch.float)
    
    edge_index = adj.nonzero(as_tuple=False).t().contiguous()
    
    return edge_index


Graph info

In [21]:
class GraphInfo:
    def __init__(self, edges: typing.Tuple[list, list], num_nodes: int):
        self.edges = edges
        self.num_nodes = num_nodes


sigma2 = 0.1
epsilon = 0.5
adj = compute_adjacency_matrix(route_distances, sigma2, epsilon)
node_indices, neighbor_indices = np.where(adj == 1)
graph = GraphInfo(
    edges=(node_indices.tolist(), neighbor_indices.tolist()),
    num_nodes=adj.shape[0],
)
print(f"number of nodes: {graph.num_nodes}, number of edges: {len(graph.edges[0])}")

edge_index=adjacency_to_edge_index(adj)
edge_index=edge_index.to(device)

number of nodes: 57, number of edges: 139


Precporcess and split data, need to edit for the case of multiple features.

In [22]:
train_size, val_size = 0.7, 0.15
def preprocess(data_array: np.ndarray, train_size: float, val_size: float):
    num_time_steps = data_array.shape[0]
    num_train = int(num_time_steps * train_size)
    num_val = int(num_time_steps * val_size)
    
    train_array = data_array[:num_train]
    val_array = data_array[num_train:num_train + num_val]
    test_array = data_array[num_train + num_val:]
    
    return train_array, val_array, test_array

# Function to average over rows in the data
def average_over_rows(data, split):
    # Check the number of dimensions in the input data
    if data.ndim == 1:
        # If data is 1D, reshape and average
        num_elements = data.shape[0]
        num_blocks = num_elements // split
        averaged_data = np.mean(data[:num_blocks * split].reshape(num_blocks, split), axis=1)
        
    elif data.ndim == 2:
        # If data is 2D, handle separately
        num_rows, num_cols = data.shape
        num_blocks = num_rows // split
        averaged_data = np.mean(data[:num_blocks * split].reshape(num_blocks, split, num_cols), axis=1)
        
    elif data.ndim == 3:
        # If data is 3D, handle separately
        num_rows, num_cols, num_depth = data.shape
        num_blocks = num_rows // split
        averaged_data = np.mean(data[:num_blocks * split].reshape(num_blocks, split, num_cols, num_depth), axis=1)
    return averaged_data

train, val, test = preprocess(speeds_array, train_size, val_size)

train=average_over_rows(train, split)
test=average_over_rows(test, split)
val=average_over_rows(val, split)

print(f"train set size: {train.shape}")
print(f"validation set size: {val.shape}")
print(f"test set size: {test.shape}")


train set size: (2956, 57)
validation set size: (633, 57)
test set size: (634, 57)


Time series of speeds 

In [23]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data_array, num_time_steps, forecast_horizon, multi_horizon=False):
        self.data_array = data_array
        self.input_sequence_length = num_time_steps
        self.forecast_horizon = forecast_horizon
        self.multi_horizon = multi_horizon
        self.target_offset = (
            num_time_steps
            if multi_horizon
            else num_time_steps + forecast_horizon - 1
        )
        self.target_seq_length = forecast_horizon if multi_horizon else 1
        self.targets = data_array[self.target_offset:]
        
        # Assuming mean and std are provided or calculated somewhere
        mean = np.mean(data_array, axis=(0, 1), keepdims=True)
        std = np.std(data_array, axis=(0, 1), keepdims=True)
        
        # Normalizing the inputs
        self.inputs = (data_array[:-forecast_horizon] - mean) / std

    def __len__(self):
        return len(self.inputs) - self.input_sequence_length + 1

    def __getitem__(self, idx):
        input_seq = self.inputs[idx:idx + self.input_sequence_length]
        if self.multi_horizon:
            target_seq = self.targets[idx: idx + self.target_seq_length]
        else:
            target_seq = self.targets[idx]
        
        return torch.tensor(input_seq, dtype=torch.float32), torch.tensor(target_seq, dtype=torch.float32)

def create_pytorch_dataset(data_array, input_sequence_length, forecast_horizon, batch_size, multi_horizon=False):
    dataset = TimeSeriesDataset(data_array, input_sequence_length, forecast_horizon, multi_horizon)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


train_dataset, val_dataset = (
    create_pytorch_dataset(data_array, num_time_steps, forecast_horizon, batch_size=batch_size)
    for data_array in [train, val]
)

test_dataset = create_pytorch_dataset(
    test,
    num_time_steps,
    forecast_horizon,
    batch_size=50)


print(f"Train dataset size: {len(train_dataset.dataset)}")
print(f"Test dataset size: {len(test_dataset.dataset)}")
print(f"Validation dataset size: {len(test_dataset.dataset)}")

Train dataset size: 2924
Test dataset size: 602
Validation dataset size: 602


Making a Neural Network, need to fix architecture.

In [24]:
if multi_horizon:
    num_predicted=forecast_horizon
else:
    num_predicted=1
 
class TemporalGatedConv1d(nn.Module):   # Opperates on (batch_size, in_channels, sequence_length) shape
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        super(TemporalGatedConv1d, self).__init__()
        self.conv_gate = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
        self.conv_linear = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
        
    def forward(self, x):
        gate_output = torch.sigmoid(self.conv_gate(x))
        linear_output = self.conv_linear(x)
        gated_output = gate_output * F.relu(linear_output)
        return gated_output

class SpatialGraphConv(nn.Module):   
    def __init__(self, in_channels, out_channels):
        super(SpatialGraphConv, self).__init__()
        if Chebyshev:
            self.conv = ChebConv(in_channels, out_channels, K=2)   
        else:
            self.conv = GCNConv(in_channels, out_channels)  
    def forward(self, x):
        return F.relu(self.conv(x, edge_index)) 

class TemporalGraphConvNet(nn.Module):
    def __init__(self):
        super(TemporalGraphConvNet, self).__init__()
        self.temporal_gated_conv1 = TemporalGatedConv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        self.spatial_graph_conv = SpatialGraphConv(in_channels=64, out_channels=16)
        
        self.spatial_graph_conv2 = SpatialGraphConv(in_channels=1, out_channels=64)
        self.temporal_gated_conv2 = TemporalGatedConv1d(in_channels=16, out_channels=64, kernel_size=3)
        self.temporal_gated_conv3 = TemporalGatedConv1d(in_channels=64, out_channels=64, kernel_size=3)
        
        self.attention1=torch.nn.MultiheadAttention(num_features,1)  #Attention opperates on (seq_length, batch_size, num_features) shape
        self.attention2=torch.nn.MultiheadAttention(64,4)

        self.dropout = nn.Dropout(p=0.2)
        self.linear_layer1 = nn.Linear(64*num_time_steps, 100)
        self.linear_layer2 = nn.Linear(100, num_predicted*num_features)
    

    def forward(self, x):
        num_features=num_feature
        if num_features == 1:
            batch_size, num_time_steps, num_nodes = x.size()
        else:
            batch_size, num_time_steps, num_nodes, num_features = x.size()


        if Attention == True:
            x = x.view(num_time_steps, batch_size  * num_nodes,num_features)
            x,_ = self.attention1(x,key=x,value=x) 
            
            x = x.view(num_time_steps*batch_size  , num_nodes,1)
            x = self.spatial_graph_conv2(x)  # 

            x = x.view(num_time_steps, batch_size  * num_nodes,64)
            x,_= self.attention2(x,key=x,value=x)  
        else:
            x = x.view(batch_size  * num_nodes,num_features, num_time_steps)
            x = self.temporal_gated_conv1(x)  

            x = x.view(batch_size*num_time_steps, num_nodes,64)  
            x = self.spatial_graph_conv(x)  # 

            x = x.view(-1, 16, num_time_steps) 
            x = self.temporal_gated_conv2(x)  

            #x = x.view(batch_size*num_time_steps, num_nodes,64)  
            #x = self.spatial_graph_conv2(x)  # 

            #x = x.view(-1, 64, num_time_steps) 
            #x  = self.temporal_gated_conv3(x)  

        # Linear layers
        x = x.view(batch_size*num_nodes,64*num_time_steps)
        x = F.relu(self.linear_layer1(x))  
        
        x = self.dropout(x) 
        x = self.linear_layer2(x)  
        
        if num_features == 1:
            x = x.view(batch_size,num_predicted,num_nodes ) 
        else:
            x = x.view(batch_size,num_predicted,num_nodes,num_features ) 
        return x
    

Initialize network

In [25]:
model = TemporalGraphConvNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
validation_losses=[]
train_losses=[]

Defining training

In [26]:
def train(model, train_loader):
    model.train()
    total_loss = 0.0
    i=0
    for data, targets in train_loader:
        i=i+1
        data, targets = data.to(device), targets.to(device) 
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        train_losses.append(loss.item()/len(train_loader.dataset))    
    return (total_loss )/i

Define test epoch

In [27]:
def test(model, test_loader):
    model.eval()
    test_loss = 0.0
    i=0
    with torch.no_grad():
        for data, targets in test_loader:
            i=i+1
            data, targets = data.to(device), targets.to(device)  
            outputs = model(data)
            test_loss += criterion(outputs, targets).item() 
    return test_loss/i

In [28]:
def test_final(model, test_loader):
    model.eval()
    test_loss = 0.0
    i=0
    with torch.no_grad():
        for data, targets in test_loader:
            i=i+1
            data, targets = data.to(device), targets.to(device)  
            outputs = model(data)
            test_loss += criterion(outputs, targets).item() 
    
    speed_difference=(test_loss)/i
    print(f'Final average speed differnce [{speed_difference:.2f}]')



Define train with early stopping, to find best hyperparameters.

In [29]:
def train_with_early_stopping(model, train_loader, val_loader):
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader)
        val_loss = test(model, val_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early stopping logic

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            # Save the best model
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1
            print(f"No improvment for : {epochs_no_improve} steps")
            if epochs_no_improve == patience:
                print('Early stopping')
                break
    model.load_state_dict(torch.load('best_model.pth'))

Running, loss is currently set to Mean Squared error.

In [30]:
print(f"Initial Test loss: {test(model, test_dataset)}")
train_with_early_stopping(model, train_dataset, val_dataset)
test_final(model, test_dataset)

C:\Users\necad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([50, 57])) that is different to the input size (torch.Size([50, 1, 57])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\Users\necad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([2, 57])) that is different to the input size (torch.Size([2, 1, 57])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\Users\necad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\

Initial Test loss: 58.42518380972055


C:\Users\necad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([12, 57])) that is different to the input size (torch.Size([12, 1, 57])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\Users\necad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([9, 57])) that is different to the input size (torch.Size([9, 1, 57])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch [1/1000], Train Loss: 16.8216, Val Loss: 8.9971
Epoch [2/1000], Train Loss: 9.4549, Val Loss: 8.9075
Epoch [3/1000], Train Loss: 9.3102, Val Loss: 8.5442
Epoch [4/1000], Train Loss: 9.2504, Val Loss: 8.6146
No improvment for : 1 steps
Epoch [5/1000], Train Loss: 9.2393, Val Loss: 8.6231
No improvment for : 2 steps
Epoch [6/1000], Train Loss: 9.1788, Val Loss: 8.3800
Epoch [7/1000], Train Loss: 9.2156, Val Loss: 8.4507
No improvment for : 1 steps
Epoch [8/1000], Train Loss: 9.1766, Val Loss: 8.4011
No improvment for : 2 steps
Epoch [9/1000], Train Loss: 9.1231, Val Loss: 8.4469
No improvment for : 3 steps
Epoch [10/1000], Train Loss: 9.1318, Val Loss: 8.3173
